In [1]:
import numpy as np
import pandas as pd
import csv_figures
from csv_figures import add_time_col
from csv_figures import annual_Norm
import matplotlib.pyplot as plt
import requests
from matplotlib.widgets import CheckButtons

In [2]:
cchrc_sharp = pd.read_excel('../../Cleaned_data/RuralAk/RuralAK_CCHRC_Sharp170_cleaned.xlsx')
cchrc_solar165 = pd.read_excel('../../Cleaned_data/RuralAk/RuralAK_CCHRC_SolarWolrd165_fixed_cleaned.xlsx')

In [3]:
cchrc_sharp_new = add_time_col(cchrc_sharp)
cchrc_solar165_new = add_time_col(cchrc_solar165)

/Users/guge/Desktop/acep-solar/new/ACEP_solar/GEGU/figures/csv_figures.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  dataframe['Date'][i] = str(dataframe['Year'][i]) + '-' + str(dataframe['Month'][i])


In [4]:
sharp_annual_values,sharp_month = annual_Norm(cchrc_sharp_new)
solar_annual_values,solar_month = annual_Norm(cchrc_solar165_new)

In [5]:
new_sharp = pd.DataFrame({'Date':sharp_month,'cchrc_sharp':sharp_annual_values})
new_solar = pd.DataFrame({'Date':solar_month,'cchrc_solar165':solar_annual_values})

In [6]:
CCHRC = pd.merge(new_sharp,new_solar,on='Date',how='outer')

In [7]:
CCHRC['Mean'] = CCHRC.mean(axis=1)
CCHRC = CCHRC.sort_values(by = ['Date'])
CCHRC

,Date,cchrc_sharp,cchrc_solar165,Mean
0,2013-07-01,1403.161765,1454.166667,1428.664216
1,2013-08-01,1430.367647,1483.712121,1457.039884
2,2013-09-01,1431.102941,1485.227273,1458.165107
3,2013-10-01,1426.323529,1481.060606,1453.692068
4,2013-11-01,1409.411765,1462.878788,1436.145276
5,2013-12-01,1402.794118,1456.818182,1429.806150
6,2014-01-01,1402.573529,1456.628788,1429.601159
7,2014-02-01,1406.250000,1459.280303,1432.765152
8,2014-03-01,1427.573529,1477.840909,1452.707219
9,2014-04-01,1436.397059,1489.204545,1462.800802


In [16]:
# Get the data from the PV Watts --TMY2
list_parameters = {"formt": 'JSON', "api_key": "spJFj2l5ghY5jwk7dNfVYs3JHbpR6BOGHQNO8Y9Z", "system_capacity": 4, "module_type": 0, "losses": 14.08,
              "array_type": 0, "tilt": 50, "azimuth": 180, "lat": 64.85, "lon": -147.83, "dataset": 'tmy2'}
json_response = requests.get("https://developer.nrel.gov/api/pvwatts/v6", params = list_parameters).json()
TMY2 = pd.DataFrame(data = json_response['outputs'])
# Get the data from the PV Watts --TMY2
list_parameters = {"formt": 'JSON', "api_key": "spJFj2l5ghY5jwk7dNfVYs3JHbpR6BOGHQNO8Y9Z", "system_capacity": 4, "module_type": 0, "losses": 14.08,
              "array_type": 0, "tilt": 50, "azimuth": 180, "lat": 64.85, "lon": -147.83, "dataset": 'tmy3'}
json_response = requests.get("https://developer.nrel.gov/api/pvwatts/v6", params = list_parameters).json()
TMY3 = pd.DataFrame(data = json_response['outputs'])

In [17]:
# CCHRC['TMY2'] = TMY2['ac_monthly']
# CCHRC['TMY3'] = TMY3['ac_monthly']

#Anchorage_16.drop(['TMY2_annual','TMY3_annual'],axis = 1,inplace = True)
TMY2_low = TMY2['ac_annual'][2]*0.95/4
TMY2_high = TMY2['ac_annual'][2]*1.05/4

TMY3_low = TMY3['ac_annual'][2]*0.95/4
TMY3_high = TMY3['ac_annual'][2]*1.05/4

CCHRC['TMY2_low'] = TMY2_low
CCHRC['TMY2_high'] = TMY2_high
CCHRC['TMY3_low'] = TMY3_low
CCHRC['TMY3_high'] = TMY3_high

In [18]:
%matplotlib qt5
fig, ax = plt.subplots(figsize = (15, 8))
x = CCHRC['Date']


y_D = CCHRC['cchrc_sharp']
l0, = ax.plot(x ,y_D,label='cchrc_sharp')

y_L = CCHRC['cchrc_solar165']
l1, = ax.plot(x ,y_L,label='cchrc_solar165')


y_mean = CCHRC['Mean']
l2, = ax.plot(x,y_mean,label='Mean')


y_TMY2_low = CCHRC['TMY2_low']
l3, = ax.plot(x,y_TMY2_low,'--',label = 'TMY2_low')

y_TMY2_high = CCHRC['TMY2_high']
l4, = ax.plot(x,y_TMY2_high,'--',label = 'TMY2_high')

y_TMY3_low = CCHRC['TMY3_low']
l5, = ax.plot(x,y_TMY3_low,'-.',label = 'TMY3_low')

y_TMY3_high = CCHRC['TMY3_high']
l6, = ax.plot(x,y_TMY3_high,'-.',label = 'TMY3_high')


ax.set_title("Rolling 12-Month Average of Produced Power, Over Time", size = 18)
ax.set_xlabel("Month", size = 14)
ax.set_ylabel("AC Power Produced (kWh) per DC Power Installed (kW)", size = 14)
ax.legend()


plt.subplots_adjust(left=0.2)
lines = [l0, l1, l2, l3, l4, l5, l6]

# Make checkbuttons with all plotted lines with correct visibility
rax = plt.axes([0.005, 0.5, 0.15, 0.4])
labels = [str(line.get_label()) for line in lines]
visibility = [line.get_visible() for line in lines]
check = CheckButtons(rax, labels, visibility)


def func(label):
    index = labels.index(label)
    lines[index].set_visible(not lines[index].get_visible())
    plt.draw()

check.on_clicked(func)


# Some 

plt.show()